In [1]:
from utils import utils,learner
from utils import PlantPathologyDataset as dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.models as models
import numpy as np

In [2]:
dtype = torch.FloatTensor
fixed_model_base = nn.Sequential(
    nn.Conv2d(3, 5, kernel_size=5, stride=1),
    nn.ReLU(inplace=True),
    nn.BatchNorm2d(5),
    nn.MaxPool2d(2, stride=2),
    learner.Flatten(),
    nn.Linear(129690, 1024),
    nn.ReLU(inplace=True),
    nn.Linear(1024, 12),
)

fixed_model = fixed_model_base.type(dtype)
resnet = models.resnet18(
    True)

loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.SGD(resnet.parameters(), lr = 0.1)
#

In [3]:
def subtractMeanImage(sample):
    mean = np.load("meanImage.npy",allow_pickle=True)
    mean = (mean*.7)
    x = sample[0]
    image =  x - mean
    sample = (image,sample[1])
    return sample

In [ ]:
thomas = False
if thomas:
    csv_file='C:\\Users\\tjtom\PycharmProjects\PlantPathology2021\\train.csv'
    root_dir='C:\\Users\\tjtom\PycharmProjects\PlantPathology2021\\train_images'
    mappings_dir='C:\\Users\\tjtom\PycharmProjects\PlantPathology2021\\labelMappings.csv'
else:
    csv_file='./train.csv'
    root_dir='./train_images'
    mappings_dir='./labelMappings.csv'
plant_dataset = dataset.PlantPathologyDataset(csv_file=csv_file,
                                              root_dir=root_dir,
                                              mappings_dir=mappings_dir,
                                              transform=subtractMeanImage)
train_data = DataLoader(plant_dataset, batch_size=5, sampler= learner.ChunkSampler(10, 0))
validation_data = DataLoader(plant_dataset, batch_size=5, sampler=learner.ChunkSampler(10, 750))
learner.train(resnet,train_data,loss_fn,optimizer,num_epochs=1, print_every=5)
acc,preds = learner.check_accuracy(resnet, validation_data)

Starting epoch 1 / 1
torch.Size([5, 3, 267, 400]) tensor([ 3, 11,  9,  9,  0])


In [5]:
acc,preds = learner.check_accuracy(resnet, validation_data)

print(preds)

Checking accuracy on validation set
Got 2 / 5 correct (40.00)
tensor([9, 0, 0, 0, 0])
